In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# Load Data

In [2]:
from preprocess_data import get_data_generators  # Ensure this file is in the same directory
train_gen, val_gen, test_gen = get_data_generators()

[ WARN:0@2.566] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


Processed images saved in skin-disease-dataset/processed_data/train
Processed images saved in skin-disease-dataset/processed_data/test
Found 741 images belonging to 8 classes.
Found 183 images belonging to 8 classes.
Found 233 images belonging to 8 classes.
Data Preprocessing & Loading Complete!
Found 741 images belonging to 8 classes.
Found 183 images belonging to 8 classes.
Found 233 images belonging to 8 classes.


# Define model

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


# Compile model

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks

In [5]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss')


# train model

In [6]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stopping, checkpoint]
)

/Users/rajeshkumar/SkinSapiens-WebApp/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - accuracy: 0.1332 - loss: 2.1060

24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 452ms/step - accuracy: 0.1335 - loss: 2.1065 - val_accuracy: 0.1475 - val_loss: 2.0798
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step - accuracy: 0.1097 - loss: 2.1192

24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 399ms/step - accuracy: 0.1102 - loss: 2.1191 - val_accuracy: 0.1366 - val_loss: 2.0700
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.1244 - loss: 2.0945

24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 405ms/step - accuracy: 0.1247 - loss: 2.0947 - val_accuracy: 0.1475 - val_loss: 2.0661
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 411ms/step - accuracy: 0.1483 - loss: 2.0975 - val_accuracy: 0.1421 - val_loss: 2.0806
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 396ms/step - accuracy: 0.1215 - loss: 2.1144 - val_accuracy: 0.1475 - val_loss: 2.0759
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 395ms/step - accuracy: 0.1422 - loss: 2.1042 - val_accuracy: 0.1475 - val_loss: 2.0710
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 392ms/step - accuracy: 0.1337 - loss: 2.0981 - val_accuracy: 0.1475 - val_loss: 2.0763
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 406ms/step - accuracy: 0.1324 - loss: 2.1091 - val_accuracy: 0.1475 - val_loss: 2.0677


# Evaluate Model

In [7]:
eval_results = model.evaluate(test_gen)
print(f"Test Loss: {eval_results[0]:.4f}, Test Accuracy: {eval_results[1]:.4f}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 290ms/step - accuracy: 0.0643 - loss: 2.0659
Test Loss: 2.0663, Test Accuracy: 0.1459


# Save Final Model

In [ ]:
model.save("final_skin_model.h5")
print("Model training complete and saved!")